In [1]:
# if (!require("BiocManager", quietly = TRUE))
#     install.packages("BiocManager")
# BiocManager::install(version = "3.10")
# BiocManager::install("randomForest")

In [2]:
setwd('/Users/alexis/University of North Carolina at Chapel Hill/Rager, Julia - CEMALB_DataAnalysisPM/Projects/P1002. Ecig Metabolomics NELF/P1002.3. Analyses/P1002.3.2. Smoking Status Prediction/Input')
Output = ('/Users/alexis/University of North Carolina at Chapel Hill/Rager, Julia - CEMALB_DataAnalysisPM/Projects/P1002. Ecig Metabolomics NELF/P1002.3. Analyses/P1002.3.1. Smoking Status Prediction/Output')
cur_date = "022422"

library(tidyverse)
library(caret)
library(e1071)
library(Hmisc)
library(glmnet)
library(pROC)
library(randomForest)

#reading in file
molecular_feature_df_w_Ecig_40 = read_csv("20220217_mf_qrilc_w40.csv") %>%
     rename(Smoking_Status = sample)
molecular_feature_df_wo_Ecig_40 = read_csv("20220217_mf_qrilc_wo40.csv") %>%
     rename(Smoking_Status = sample)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.0.6     ✔ dplyr   1.0.4
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift


Loading required package: survival


Attaching package: ‘survival’


The following object is masked from ‘package:caret’:

    cluster


Loading required package: Formula


Attaching package: ‘Hmisc’


The following object is masked from ‘package:e1071’:

    impute


The following objects are masked from ‘package:dplyr’:

    src, summarize


The following objects are masked from ‘package:base’:

    format.pval, units


Loading required package: Matrix


Attaching package: ‘Mat

In [3]:
head(molecular_feature_df_w_Ecig_40)

Smoking_Status,MF_2,MF_3,MF_4,MF_5,MF_6,MF_7,MF_11,MF_12,MF_13,⋯,MF_62853,MF_63431,MF_63701,MF_64025,MF_64936,MF_65532,MF_68286,MF_73340,MF_77165,MF_78976
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
NS_070,19.86186,17.86943,19.21394,18.02809,17.46636,18.34174,18.41799,19.31361,19.46777,⋯,19.06974,19.10260,17.17255,16.92465,17.09794,19.06203,19.13556,16.93182,17.55702,17.05593
NS_110,21.08622,22.58769,21.05378,17.93584,17.33098,20.41479,20.37485,21.12876,21.01488,⋯,18.02560,17.82667,19.37426,16.23450,19.06421,18.06379,19.39103,16.72112,19.27343,19.30881
NS_143,13.12099,14.44659,13.46827,16.51009,14.93820,15.66299,15.72731,13.78698,15.18975,⋯,19.14095,19.02411,19.56175,17.17120,16.97302,19.49121,17.09470,15.56214,19.55038,19.54912
NS_152,21.60361,23.22761,21.99010,16.22547,17.59020,21.53908,21.14531,22.07857,21.97502,⋯,19.28751,19.54886,16.78848,16.46750,19.47635,19.44477,16.99617,16.09531,19.28810,19.32788
NS_187,19.43885,17.93876,19.68977,16.73360,17.88844,17.81758,17.64979,19.64269,19.32233,⋯,16.70042,17.85969,18.97406,18.18813,18.50336,19.19085,19.41757,16.88993,19.35678,18.95550
NS_196,22.53459,24.36662,22.77595,16.69956,16.38411,22.55912,17.30008,22.80146,22.09338,⋯,16.86999,15.82637,19.46467,16.97149,17.46699,16.32598,15.62938,17.37207,15.84334,17.82000


In [4]:
#first making smoking status column binary denoted by 0 = non smoker and 1 = ecig user
molecular_feature_df_w_Ecig_40 = molecular_feature_df_w_Ecig_40 %>%
    #grepl is synonymous with "contains"
    #necessary to put it into factor and relevel, so algorithm understands which data is considered baseline
    mutate(Smoking_Status = relevel(factor(ifelse(grepl("Ecig", Smoking_Status), 1, 0)), ref = "0"))

molecular_feature_df_wo_Ecig_40 = molecular_feature_df_wo_Ecig_40 %>%
    mutate(Smoking_Status = relevel(factor(ifelse(grepl("Ecig", Smoking_Status), 1, 0)), ref = "0"))

mf_df_w_Ecig_40_wo_nicotine = molecular_feature_df_w_Ecig_40 %>%
    #removing nicotine and cotinine
    select(-c(MF_97, MF_5))

mf_df_wo_Ecig_40_wo_nicotine = molecular_feature_df_wo_Ecig_40 %>%
    select(-c(MF_97, MF_5))

head(molecular_feature_df_w_Ecig_40)

Smoking_Status,MF_2,MF_3,MF_4,MF_5,MF_6,MF_7,MF_11,MF_12,MF_13,⋯,MF_62853,MF_63431,MF_63701,MF_64025,MF_64936,MF_65532,MF_68286,MF_73340,MF_77165,MF_78976
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,19.86186,17.86943,19.21394,18.02809,17.46636,18.34174,18.41799,19.31361,19.46777,⋯,19.06974,19.10260,17.17255,16.92465,17.09794,19.06203,19.13556,16.93182,17.55702,17.05593
0,21.08622,22.58769,21.05378,17.93584,17.33098,20.41479,20.37485,21.12876,21.01488,⋯,18.02560,17.82667,19.37426,16.23450,19.06421,18.06379,19.39103,16.72112,19.27343,19.30881
0,13.12099,14.44659,13.46827,16.51009,14.93820,15.66299,15.72731,13.78698,15.18975,⋯,19.14095,19.02411,19.56175,17.17120,16.97302,19.49121,17.09470,15.56214,19.55038,19.54912
0,21.60361,23.22761,21.99010,16.22547,17.59020,21.53908,21.14531,22.07857,21.97502,⋯,19.28751,19.54886,16.78848,16.46750,19.47635,19.44477,16.99617,16.09531,19.28810,19.32788
0,19.43885,17.93876,19.68977,16.73360,17.88844,17.81758,17.64979,19.64269,19.32233,⋯,16.70042,17.85969,18.97406,18.18813,18.50336,19.19085,19.41757,16.88993,19.35678,18.95550
0,22.53459,24.36662,22.77595,16.69956,16.38411,22.55912,17.30008,22.80146,22.09338,⋯,16.86999,15.82637,19.46467,16.97149,17.46699,16.32598,15.62938,17.37207,15.84334,17.82000


The goal of this analysis is to use molecular features [continuous] to predict smoking staus (dichotomous). This will be done using Support Vector Machine (SVM) based on a logistic regression model for classification and Random Forest.

# Random Forest

- an ensemble learning method operating by constructing a multitude of decision trees at training time, which uses multiple methods to obtain a better predictive performance and includes bagging and random forest
- algorithm uses a bootstrop aggregation, to reduce overfitting the training datset but only a subset of the features are used hence decorrelation of predictors

In [5]:
rf_classification = function(dataset, outcome, pred_outcome){
    #setting for reproducibility
    set.seed(12)
    #splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) #5 fold CV

    metrics = data.frame()
    variable_importance_df = data.frame()
    
    for (i in 1:length(dataset_index)){
        data_train = dataset[-dataset_index[[i]],]
        data_test = dataset[dataset_index[[i]],]

        ntree_values = c(50, 250, 500) #number of trees (not sure if we wanted something different)
        p = dim(dataset)[2] - 1 #number of variables in dataset
        mtry_values = c(p/2, sqrt(p), p)

        #will use ntree and mtry values to determine which combination yields the smallest MSE
        reg_rf_pred_tune = list()
        rf_OOB_errors = list()
        rf_error_df = data.frame()
        for (j in 1:length(ntree_values)){
            for (k in 1:length(mtry_values)){
                reg_rf_pred_tune[[k]] = randomForest(as.formula(paste0(outcome, "~.")), data = data_train, 
                                                     ntree = ntree_values[j], mtry = mtry_values[k])
                rf_OOB_errors[[k]] = data.frame("Tree Number" = ntree_values[j], "Variable Number" = mtry_values[k], 
                                       "OOB_errors" = reg_rf_pred_tune[[k]]$err.rate[ntree_values[j],1])
                rf_error_df = rbind(rf_error_df, rf_OOB_errors[[k]])
            }
        }

        #finding the lowest OOB error using best number of predictors at split and refitting OG tree
        best_oob_errors <- which(rf_error_df$OOB_errors == min(rf_error_df$OOB_errors))

        reg_rf <- randomForest(as.formula(paste0(outcome, "~.")), data = data_train,
        #need to go back and figure out how the number of trees/ variables make a difference even though
        #the errors are the same!!!!
                               ntree = rf_error_df$Tree.Number[min(best_oob_errors)], 
                               mtry = rf_error_df$Variable.Number[min(best_oob_errors)])

        #predicting on test set
        data_test[[pred_outcome]] = predict(reg_rf, newdata = data_test, type = "response")


        matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                     positive = "1")

        #calculating AUC
        auc = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))
        #extracting accuracy, sens, spec, PPV to take mean later
        matrix_values = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:3])), auc)
        
        #extracting variable importance
        var_importance_values = data.frame(importance(reg_rf)) %>%
            rownames_to_column(var = "MF")
        variable_importance_df = rbind(variable_importance_df, var_importance_values)
   
        #adding values to df
        metrics = rbind(metrics, matrix_values)
    }
    
    #taking averages/sd 
    metrics = metrics %>%
        summarise(`Balanced Accuracy` = mean(Balanced.Accuracy), Sensitivity = mean(Sensitivity), 
              Specificity = mean(Specificity), PPV = mean(Pos.Pred.Value), AUC = mean(auc))
    
    variable_importance_df = variable_importance_df %>%
        group_by(MF) %>%
        summarise(MeanDecreaseGini = mean(MeanDecreaseGini)) %>%
        #sorting by most important variables
        arrange(-MeanDecreaseGini)
  
    #return training set, matrix and variable importance values
    return(list(data_train, metrics, variable_importance_df))

}

#calling fn
rf_values_w_Ecig_40 = rf_classification(molecular_feature_df_w_Ecig_40, "Smoking_Status", "pred_Smoking_Status")
rf_values_wo_Ecig_40 = rf_classification(molecular_feature_df_wo_Ecig_40, "Smoking_Status", "pred_Smoking_Status")
rf_values_w_Ecig_40_wo_nicotine = rf_classification(mf_df_w_Ecig_40_wo_nicotine, "Smoking_Status", "pred_Smoking_Status")
rf_values_wo_Ecig_40_wo_nicotine = rf_classification(mf_df_wo_Ecig_40_wo_nicotine, "Smoking_Status", "pred_Smoking_Status")

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting leve

In [6]:
#viewing results
rf_confusion_matrix = data.frame(Model = c(rep(c("Original RF"), times = 4)), Dataset = c(rep(c("Ecig_40", "no_Ecig_40"), times = 2)),
                                 Removed.Variables = c(rep(c("None"), times = 2), rep(c("Nicotine & Cotinine"), times = 2)),
                                 rbind(rf_values_w_Ecig_40[[2]], rf_values_wo_Ecig_40[[2]], rf_values_w_Ecig_40_wo_nicotine[[2]], rf_values_wo_Ecig_40_wo_nicotine[[2]]))
rf_confusion_matrix

#viewing most significant features
head(rf_values_w_Ecig_40[[3]])
head(rf_values_wo_Ecig_40[[3]]) 
head(rf_values_w_Ecig_40_wo_nicotine[[3]])
head(rf_values_wo_Ecig_40_wo_nicotine[[3]]) 

Model,Dataset,Removed.Variables,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Original RF,Ecig_40,None,0.9047619,0.8428571,0.9666667,0.9600000,0.9047619
Original RF,no_Ecig_40,None,0.9047619,0.8428571,0.9666667,0.9666667,0.9047619
Original RF,Ecig_40,Nicotine & Cotinine,0.6185714,0.6571429,0.5800000,0.6883333,0.6185714
Original RF,no_Ecig_40,Nicotine & Cotinine,0.6657143,0.7714286,0.5600000,0.6961905,0.6657143


MF,MeanDecreaseGini
<chr>,<dbl>
MF_97,7.0542406
MF_5,4.6837830
MF_292,0.2770581
MF_3345,0.2480095
MF_1713,0.1876070
MF_4803,0.1654220


MF,MeanDecreaseGini
<chr>,<dbl>
MF_97,7.0597273
MF_5,4.0815330
MF_13899,0.2952068
MF_4858,0.1918325
MF_5547,0.1915535
MF_292,0.1884780


MF,MeanDecreaseGini
<chr>,<dbl>
MF_292,0.8834351
MF_7221,0.5365767
MF_3345,0.4643966
MF_5547,0.4396756
MF_1324,0.3772763
MF_1713,0.3212409


MF,MeanDecreaseGini
<chr>,<dbl>
MF_13899,0.8453392
MF_5547,0.6688683
MF_8500,0.3760636
MF_2213,0.2875837
MF_292,0.2735644
MF_1324,0.2355759


Rerunning RF model with noise variables to determine, which predictors are significant above the background noise. 

In [10]:
train_vars_noise_w_Ecig_40 = rf_values_w_Ecig_40[[1]]
train_vars_noise_wo_Ecig_40 = rf_values_wo_Ecig_40[[1]]
train_vars_noise_w_Ecig_40_wo_nicotine = rf_values_w_Ecig_40_wo_nicotine[[1]]
train_vars_noise_wo_Ecig_40_wo_nicotine = rf_values_wo_Ecig_40_wo_nicotine[[1]]

noise_df = function(train_vars_noise){
    set.seed(7)
    # Add random noise predictors as an additional method to evaluate model performance
    # Adding a column that contains randomly shuffled values from one of the molecules; sampling with replacement
    train_vars_noise$noise1 = sample(train_vars_noise[[colnames(train_vars_noise[3])]], replace = TRUE) 
    train_vars_noise$noise2 = sample(train_vars_noise[[colnames(train_vars_noise[4])]], replace = TRUE)
    train_vars_noise$noise3 = sample(train_vars_noise[[colnames(train_vars_noise[5])]], replace = TRUE)
    train_vars_noise$noise4 = sample(train_vars_noise[[colnames(train_vars_noise[6])]], replace = TRUE)
    train_vars_noise$noise5 = sample(train_vars_noise[[colnames(train_vars_noise[7])]], replace = TRUE)
    
    return(train_vars_noise)
}

#calling fn
noise_training_rf_dataset_w_Ecig_40 = noise_df(train_vars_noise_w_Ecig_40)
noise_training_rf_dataset_wo_Ecig_40 = noise_df(train_vars_noise_wo_Ecig_40)
noise_training_rf_dataset_w_Ecig_40_wo_nicotine = noise_df(train_vars_noise_w_Ecig_40_wo_nicotine)
noise_training_rf_dataset_wo_Ecig_40_wo_nicotine = noise_df(train_vars_noise_wo_Ecig_40_wo_nicotine)
head(noise_training_rf_dataset_w_Ecig_40)

Smoking_Status,MF_2,MF_3,MF_4,MF_5,MF_6,MF_7,MF_11,MF_12,MF_13,⋯,MF_65532,MF_68286,MF_73340,MF_77165,MF_78976,noise1,noise2,noise3,noise4,noise5
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,19.86186,17.86943,19.21394,18.02809,17.46636,18.34174,18.41799,19.31361,19.46777,⋯,19.06203,19.13556,16.93182,17.55702,17.05593,22.95603,21.55396,19.64002,19.91744,22.63653
0,21.08622,22.58769,21.05378,17.93584,17.33098,20.41479,20.37485,21.12876,21.01488,⋯,18.06379,19.39103,16.72112,19.27343,19.30881,23.00581,22.91377,15.77071,17.88844,23.46244
0,13.12099,14.44659,13.46827,16.51009,14.93820,15.66299,15.72731,13.78698,15.18975,⋯,19.49121,17.09470,15.56214,19.55038,19.54912,22.86175,22.54138,20.63665,17.31698,22.06629
0,19.43885,17.93876,19.68977,16.73360,17.88844,17.81758,17.64979,19.64269,19.32233,⋯,19.19085,19.41757,16.88993,19.35678,18.95550,24.32335,22.14102,16.08028,17.71179,23.15163
0,22.53459,24.36662,22.77595,16.69956,16.38411,22.55912,17.30008,22.80146,22.09338,⋯,16.32598,15.62938,17.37207,15.84334,17.82000,22.73364,20.20755,21.28516,26.11076,22.61793
0,21.50216,23.34553,21.53633,17.93372,24.26751,21.71412,21.32122,21.66214,21.71152,⋯,16.71579,18.52501,17.61326,17.74147,16.76017,22.58769,19.91826,16.08028,15.36472,22.43186


In [11]:
#calling rf fn
noise_rf_values_w_Ecig_40 = rf_classification(noise_training_rf_dataset_w_Ecig_40, "Smoking_Status", "pred_Smoking_Status")
noise_rf_values_wo_Ecig_40 = rf_classification(noise_training_rf_dataset_wo_Ecig_40, "Smoking_Status", "pred_Smoking_Status")
noise_rf_values_w_Ecig_40_wo_nicotine = rf_classification(noise_training_rf_dataset_w_Ecig_40_wo_nicotine, "Smoking_Status", "pred_Smoking_Status")
noise_rf_values_wo_Ecig_40_wo_nicotine = rf_classification(noise_training_rf_dataset_wo_Ecig_40_wo_nicotine, "Smoking_Status", "pred_Smoking_Status")

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting leve

In [12]:
#viewing results
noise_rf_confusion_matrix = data.frame(Model = c(rep(c("RF w/ Noise"), times = 4)), Dataset = c(rep(c("Ecig_40", "no_Ecig_40"), times = 2)), 
                                       Removed.Variables = c(rep(c("None"), times = 2), rep(c("Nicotine & Cotinine"), times = 2)), 
                                       rbind(noise_rf_values_w_Ecig_40[[2]], noise_rf_values_wo_Ecig_40[[2]],
                                                                               noise_rf_values_w_Ecig_40_wo_nicotine[[2]], noise_rf_values_wo_Ecig_40_wo_nicotine[[2]]))
noise_rf_confusion_matrix

#filtering for most significant features
head(noise_rf_values_w_Ecig_40[[3]])
head(noise_rf_values_wo_Ecig_40[[3]]) 
head(noise_rf_values_w_Ecig_40_wo_nicotine[[3]])
head(noise_rf_values_wo_Ecig_40_wo_nicotine[[3]]) 

Model,Dataset,Removed.Variables,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
RF w/ Noise,Ecig_40,None,0.8783333,0.9266667,0.83,0.8809524,0.8783333
RF w/ Noise,no_Ecig_40,None,0.8550000,0.8000000,0.91,0.9200000,0.8550000
RF w/ Noise,Ecig_40,Nicotine & Cotinine,0.5600000,0.5800000,0.54,0.6600000,0.5600000
RF w/ Noise,no_Ecig_40,Nicotine & Cotinine,0.6950000,0.7600000,0.63,0.7109524,0.6950000


MF,MeanDecreaseGini
<chr>,<dbl>
MF_97,2.6118845
MF_5,1.8559144
MF_292,0.7185532
MF_7221,0.3767431
MF_29367,0.2427987
MF_6955,0.1876891


MF,MeanDecreaseGini
<chr>,<dbl>
MF_5,3.5361642
MF_97,2.6207010
MF_13899,0.8559147
MF_292,0.3441422
MF_4803,0.2941377
MF_5782,0.2315247


MF,MeanDecreaseGini
<chr>,<dbl>
MF_292,0.5308698
MF_3345,0.3398274
MF_7221,0.2814058
MF_6955,0.2727071
MF_1522,0.2459021
MF_34412,0.1909755


MF,MeanDecreaseGini
<chr>,<dbl>
MF_13899,1.0893093
MF_292,0.6250473
MF_4803,0.5028013
MF_5547,0.3502884
MF_9367,0.2907885
MF_8500,0.2225841


# Support Vector Machine
- supervised learning models that can predict continuous (regression) or grouped/dichotomous (classification) outcomes
- predictors by projecting them onto a high dimensional space and uses kernels to make the data more separable (unfortunately makes interpretability of model results more difficult)
- does a better job at handling a large number of predictors since p > n

In [13]:
#model errors are calculated to assess the predictive accuracy of the model
svm_classification = function(dataset, model, outcome, pred_outcome, gamma_values, cost_values, elsilon_values){
  
    #setting seed for reproducibility
    set.seed(12)
    
    #splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) #5 fold CV
    metrics = data.frame()
    for (i in 1:length(dataset_index)){
        data_train = dataset[-dataset_index[[i]],]
        data_test = dataset[dataset_index[[i]],]

        #now pruning parameters (based on the training dataset to prevent overfitting)
        svr_tune <- tune(svm, as.formula(paste0(outcome, "~.")), data = data_train, kernel = model, 
                       ranges = list(elsilon = elsilon_values, cost = cost_values, gamma = gamma_values)) 

        #choosing best model
        best_svm <- svr_tune$best.model

        #predicting with tuned parameters 
        data_test[[pred_outcome]] <- predict(best_svm, newdata = data_test, type = "response")

        matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                 positive = "1")
 
        #calculating AUC
        auc = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))
        #extracting accuracy, sens, spec, PPV to take mean later
        matrix_values = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:3])), auc)
   
        #adding values to df
        metrics = rbind(metrics, matrix_values)
  }
  
  #taking averages/sd 
  metrics = metrics %>%
    summarise(`Balanced Accuracy` = mean(Balanced.Accuracy), Sensitivity = mean(Sensitivity), 
              Specificity = mean(Specificity), PPV = mean(Pos.Pred.Value), AUC = mean(auc))
  
  return(metrics)
}

In [ ]:
#calling fn
svm_w_Ecig_40 = svm_classification(molecular_feature_df_w_Ecig_40, "linear", "Smoking_Status", "pred_Smoking_Status",
                                            0.035, 1:5, seq(0,1,0.2))
svm_wo_Ecig_40 = svm_classification(molecular_feature_df_wo_Ecig_40, "linear", "Smoking_Status", "pred_Smoking_Status",
                                            0.035, 1:5, seq(0,1,0.2))
svm_w_Ecig_40_wo_nicotine = svm_classification(mf_df_w_Ecig_40_wo_nicotine, "linear", "Smoking_Status", "pred_Smoking_Status",
                                            0.035, 1:5, seq(0,1,0.2))
svm_wo_Ecig_40_wo_nicotine = svm_classification(mf_df_wo_Ecig_40_wo_nicotine, "linear", "Smoking_Status", "pred_Smoking_Status",
                                            0.035, 1:5, seq(0,1,0.2))

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting leve

In [12]:
#creating one combined df
svm_final_df = data.frame(rbind(svm_w_Ecig_40, svm_wo_Ecig_40, svm_w_Ecig_40_wo_nicotine, svm_wo_Ecig_40_wo_nicotine))

svm_final_df = data.frame(Model = c(rep(c("Original SVM"), times = 4)), Dataset = c(rep(c("Ecig_40", "no_Ecig_40"), times = 2)),
                          Removed.Variables = c(rep(c("None"), times = 2), rep(c("Nicotine & Cotinine"), times = 2)),, svm_final_df)
svm_final_df

Model,Dataset,Kernel,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Original SVM,Ecig_40,linear,0.7614286,0.8095238,0.7133333,0.8044444,0.7614286
Original SVM,Ecig_40,radial,0.5000000,1.0000000,0.0000000,0.5424242,0.5000000
Original SVM,Ecig_40,polynomial,0.6057143,0.9047619,0.3066667,0.6187013,0.6057143
Original SVM,no_Ecig_40,linear,0.8100000,0.8000000,0.8200000,0.8666667,0.8100000
Original SVM,no_Ecig_40,radial,0.5000000,1.0000000,0.0000000,0.5348485,0.5000000
Original SVM,no_Ecig_40,polynomial,0.6057143,0.9047619,0.3066667,0.6070996,0.6057143


# RF Variable Selection w/ SVM

Calculating the average noise rank to determine, which predictors ranked higher than the average noise rank. Then filtering the original datasets for those predictors and rerunning SVM.

In [ ]:
#calculating avg noise rank out of all variables
noise_importance_values_w_Ecig_40 = noise_rf_values_w_Ecig_40[[3]] %>%
    column_to_rownames(var = "MF")
noise_importance_values_wo_Ecig_40 = noise_rf_values_wo_Ecig_40[[3]] %>%
    column_to_rownames(var = "MF")
noise_importance_values_w_Ecig_40_wo_nicotine = noise_rf_values_w_Ecig_40_wo_nicotine[[3]] %>%
    column_to_rownames(var = "MF")
noise_importance_values_wo_Ecig_40_wo_nicotine = noise_rf_values_wo_Ecig_40_wo_nicotine[[3]] %>%
    column_to_rownames(var = "MF")

In [ ]:
avg_noise_rank = function(noise_importance_values){
    noise1rank = which(rownames(noise_importance_values) == "noise1")/nrow(noise_importance_values)
    noise2rank = which(rownames(noise_importance_values) == "noise2")/nrow(noise_importance_values)
    noise3rank = which(rownames(noise_importance_values) == "noise3")/nrow(noise_importance_values)
    noise4rank = which(rownames(noise_importance_values) == "noise4")/nrow(noise_importance_values)
    noise5rank = which(rownames(noise_importance_values) == "noise5")/nrow(noise_importance_values)

    #calculating avg mean decrease gini or background noise importance value
    avg_noise_rank = (noise1rank + noise2rank + noise3rank + noise4rank + noise5rank)/5
    
    return(avg_noise_rank)
}

#calling fn
avg_noise_rank_w_Ecig_40 = avg_noise_rank(noise_importance_values_w_Ecig_40)
avg_noise_rank_wo_Ecig_40 = avg_noise_rank(noise_importance_values_wo_Ecig_40)
avg_noise_rank_w_Ecig_40_wo_nicotine = avg_noise_rank(noise_importance_values_w_Ecig_40_wo_nicotine)
avg_noise_rank_wo_Ecig_40_wo_nicotine = avg_noise_rank(noise_importance_values_wo_Ecig_40_wo_nicotine)
avg_noise_rank_w_Ecig_40
avg_noise_rank_wo_Ecig_40

In [ ]:
last_significant_rank = function(noise_importance_values){
    for (i in 1:length(noise_importance_values$MeanDecreaseGini)){
    #if the row number (index)/# of variables is less than the avg noise rank the loop will continue
    if (i/length(noise_importance_values$MeanDecreaseGini) > avg_noise_rank){
        last_significant_rank = i - 1
        break
    }
        
    return(last_significant_rank)

}

#calling fn
last_significant_rank_w_Ecig_40 = last_significant_rank(noise_importance_values_w_Ecig_40)
last_significant_rank_wo_Ecig_40 = last_significant_rank(noise_importance_values_wo_Ecig_40)
last_significant_rank_w_Ecig_40_wo_nicotine = last_significant_rank(noise_importance_values_w_Ecig_40_wo_nicotine)
last_significant_rank_wo_Ecig_40_wo_nicotine = last_significant_rank(noise_importance_values_wo_Ecig_40_wo_nicotine)
last_significant_rank_w_Ecig_40
last_significant_rank_wo_Ecig_40

Therefore, the dataset containing Ecig_40 contains 2,098 MFs above background, while the dataset without the outlier contains 3,237. This means that the outlier dampened the significance of about 1,000 MF just by being included in the dataset. 

In [15]:
#now getting those significant MFs
significant_MFs_w_Ecig_40 = rownames(noise_importance_values_w_Ecig_40)[1:last_significant_rank_w_Ecig_40]
significant_MFs_wo_Ecig_40 = rownames(noise_importance_values_wo_Ecig_40)[1:last_significant_rank_wo_Ecig_40]
significant_MFs_w_Ecig_40_wo_nicotine = rownames(noise_importance_values_w_Ecig_40_wo_nicotine)[1:last_significant_rank_w_Ecig_40_wo_nicotine]
significant_MFs_wo_Ecig_40_wo_nicotine = rownames(noise_importance_values_wo_Ecig_40_wo_nicotine)[1:last_significant_rank_wo_Ecig_40_wo_nicotine]

#now subsetting the original dfs
rf_subset_MF_df_w_Ecig_40 = cbind(Smoking_Status = molecular_feature_df_w_Ecig_40$Smoking_Status, molecular_feature_df_w_Ecig_40[,colnames(molecular_feature_df_w_Ecig_40) %in% significant_MFs_w_Ecig_40])
rf_subset_MF_df_wo_Ecig_40 = cbind(Smoking_Status = molecular_feature_df_wo_Ecig_40$Smoking_Status, molecular_feature_df_wo_Ecig_40[,colnames(molecular_feature_df_wo_Ecig_40) %in% significant_MFs_wo_Ecig_40])
rf_subset_MF_df_w_Ecig_40_wo_nicotine = cbind(Smoking_Status = mf_df_w_Ecig_40_wo_nicotine$Smoking_Status, mf_df_w_Ecig_40_wo_nicotine[,colnames(mf_df_w_Ecig_40_wo_nicotine) %in% significant_MFs_w_Ecig_40_wo_nicotine])
rf_subset_MF_df_wo_Ecig_40_wo_nicotine = cbind(Smoking_Status = mf_df_w_Ecig_40_wo_nicotine$Smoking_Status, mf_df_wo_Ecig_40_wo_nicotine[,colnames(mf_df_wo_Ecig_40_wo_nicotine) %in% significant_MFs_wo_Ecig_40_wo_nicotine])
head(rf_subset_MF_df_w_Ecig_40)

,Smoking_Status,MF_2,MF_5,MF_11,MF_12,MF_13,MF_14,MF_15,MF_17,MF_18,⋯,MF_58765,MF_59362,MF_61188,MF_61576,MF_62137,MF_62282,MF_64025,MF_64936,MF_65532,MF_78976
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,19.86186,18.02809,18.41799,19.31361,19.46777,20.80616,18.72405,20.11458,18.49551,⋯,17.77637,19.14311,17.56748,18.64535,17.47760,18.83525,16.92465,17.09794,19.06203,17.05593
2,0,21.08622,17.93584,20.37485,21.12876,21.01488,19.71124,19.17696,19.82979,19.80685,⋯,19.31517,18.14936,17.46308,17.53162,16.35027,19.31905,16.23450,19.06421,18.06379,19.30881
3,0,13.12099,16.51009,15.72731,13.78698,15.18975,18.19551,18.84322,18.65454,17.52149,⋯,19.42314,21.24468,19.03004,19.47983,15.64893,16.80361,17.17120,16.97302,19.49121,19.54912
4,0,21.60361,16.22547,21.14531,22.07857,21.97502,17.30567,19.61857,18.35221,17.14531,⋯,18.92037,15.54892,17.10609,19.26672,17.78108,19.06662,16.46750,19.47635,19.44477,19.32788
5,0,19.43885,16.73360,17.64979,19.64269,19.32233,21.33752,19.56012,20.23552,18.70226,⋯,19.28841,19.98009,19.85844,18.88816,17.97428,19.13998,18.18813,18.50336,19.19085,18.95550
6,0,22.53459,16.69956,17.30008,22.80146,22.09338,19.51343,19.39161,20.72501,21.45596,⋯,18.50525,16.54808,19.25352,16.37276,16.06782,19.19848,16.97149,17.46699,16.32598,17.82000


In [16]:
#calling fn
subset_svm_w_Ecig_40 = svm_classification(rf_subset_MF_df_w_Ecig_40, "linear", "Smoking_Status", "pred_Smoking_Status",
                                            0.035, 1:5, seq(0,1,0.2))
subset_svm_wo_Ecig_40 = svm_classification(rf_subset_MF_df_wo_Ecig_40, "linear", "Smoking_Status", "pred_Smoking_Status",
                                            0.035, 1:5, seq(0,1,0.2))
subset_svm_w_Ecig_40_wo_nicotine = svm_classification(rf_subset_MF_df_w_Ecig_40_wo_nicotine, "linear", "Smoking_Status", "pred_Smoking_Status",
                                            0.035, 1:5, seq(0,1,0.2))
subset_svm_wo_Ecig_40_wo_nicotine = svm_classification(rf_subset_MF_df_wo_Ecig_40_wo_nicotine, "linear", "Smoking_Status", "pred_Smoking_Status",
                                            0.035, 1:5, seq(0,1,0.2))

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting leve

In [17]:
#creating one combined df
subset_svm_final_df = data.frame(rbind(subset_svm_linear_w_Ecig_40, subset_svm_linear_wo_Ecig_40, subset_svm_w_Ecig_40_wo_nicotine,
                                       subset_svm_wo_Ecig_40_wo_nicotine))

subset_svm_final_df = data.frame(Model = c(rep(c("SVM w/ RF Var Selection"), times = 4)), Dataset = c(rep(c("Ecig_40", "no_Ecig_40"), times = 2)),
                                  Removed.Variables = c(rep(c("None"), times = 2), rep(c("Nicotine & Cotinine"), times = 2)), subset_svm_final_df)
subset_svm_final_df

Model,Dataset,Kernel,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
SVM w/ RF Var Selection,Ecig_40,linear,0.8157143,0.8380952,0.7933333,0.8461111,0.8157143
SVM w/ RF Var Selection,Ecig_40,radial,0.5000000,1.0000000,0.0000000,0.5424242,0.5000000
SVM w/ RF Var Selection,Ecig_40,polynomial,0.6304762,0.8476190,0.4133333,0.6806061,0.6304762
SVM w/ RF Var Selection,no_Ecig_40,linear,0.8300000,0.8333333,0.8266667,0.8600000,0.8300000
SVM w/ RF Var Selection,no_Ecig_40,radial,0.5000000,1.0000000,0.0000000,0.5348485,0.5000000
SVM w/ RF Var Selection,no_Ecig_40,polynomial,0.6700000,0.9333333,0.4066667,0.6528571,0.6700000


In [27]:
#all results
final_df = rbind(rf_confusion_matrix, svm_final_df, subset_svm_final_df)
final_df

Model,Dataset,Kernel,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<fct>,<fct>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Original RF,Ecig_40,NA,0.9047619,0.8428571,0.9666667,0.9600000,0.9047619
Original RF,no_Ecig_40,NA,0.9047619,0.8428571,0.9666667,0.9666667,0.9047619
Original SVM,Ecig_40,linear,0.7614286,0.8095238,0.7133333,0.8044444,0.7614286
Original SVM,Ecig_40,radial,0.5000000,1.0000000,0.0000000,0.5424242,0.5000000
Original SVM,Ecig_40,polynomial,0.6057143,0.9047619,0.3066667,0.6187013,0.6057143
Original SVM,no_Ecig_40,linear,0.8100000,0.8000000,0.8200000,0.8666667,0.8100000
Original SVM,no_Ecig_40,radial,0.5000000,1.0000000,0.0000000,0.5348485,0.5000000
Original SVM,no_Ecig_40,polynomial,0.6057143,0.9047619,0.3066667,0.6070996,0.6057143
SVM w/ RF Var Selection,Ecig_40,linear,0.8157143,0.8380952,0.7933333,0.8461111,0.8157143
